In [1]:
import pandas as pd

In [86]:
tr_path='/Users/hanyhamdani/Desktop/AppliedPython/machineLearning/TitanicKaggle/train.csv'
ts_path='/Users/hanyhamdani/Desktop/AppliedPython/machineLearning/TitanicKaggle/test.csv'
train=pd.read_csv(tr_path)
test=pd.read_csv(ts_path)
train.shape,test.shape


((891, 12), (418, 11))

In [11]:
train.head()


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [12]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [79]:
pass_train_id=train.PassengerId
train.drop(['Name','Cabin','Ticket','PassengerId'],axis=1,inplace=True)
train.head()

AttributeError: 'DataFrame' object has no attribute 'PassengerId'

In [87]:
#we will remove passenger ID here so that it doesnt impact modelling

pass_test_id=test.PassengerId
test.drop(['Name','Cabin','Ticket','PassengerId'],axis=1,inplace=True)
test.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,male,34.5,0,0,7.8292,Q
1,3,female,47.0,1,0,7.0000,S
2,2,male,62.0,0,0,9.6875,Q
3,3,male,27.0,0,0,8.6625,S
4,3,female,22.0,1,1,12.2875,S


In [15]:
test.isna().sum() #checking values that we need to fill in for test data

Pclass       0
Sex          0
Age         86
SibSp        0
Parch        0
Fare         1
Embarked     0
dtype: int64

In [17]:
#for fare just replace it with mean or median

test['Fare'].fillna(test['Fare'].mean(),inplace =True)
test['Fare'].isna().any()

False

In [18]:
#Since sklearn converts all categorical data to float before encoding the model, converting all the cat data into dummy values
dummy_col=['Embarked','Sex']
test=pd.get_dummies(test,prefix=dummy_col, columns=dummy_col,drop_first=True )
test.head()


,Pclass,Age,SibSp,Parch,Fare,Embarked_Q,Embarked_S,Sex_male
0,3,34.5,0,0,7.8292,1,0,1
1,3,47.0,1,0,7.0000,0,1,0
2,2,62.0,0,0,9.6875,1,0,1
3,3,27.0,0,0,8.6625,0,1,1
4,3,22.0,1,1,12.2875,0,1,0


In [20]:
#for age, since there are 86 values, a very high value
# so predicting age values through knr

#using non null data for training to get age
non_null_tr=test[test.Age.notnull()]
non_null_tr.isna().sum()


Pclass        0
Age           0
SibSp         0
Parch         0
Fare          0
Embarked_Q    0
Embarked_S    0
Sex_male      0
dtype: int64

In [21]:
#creating x, y datas for training and testing
Xage_train=non_null_tr.drop('Age',axis=1)
Yage_train=non_null_tr.Age

In [22]:
Xage_train.head()

,Pclass,SibSp,Parch,Fare,Embarked_Q,Embarked_S,Sex_male
0,3,0,0,7.8292,1,0,1
1,3,1,0,7.0000,0,1,0
2,2,0,0,9.6875,1,0,1
3,3,0,0,8.6625,0,1,1
4,3,1,1,12.2875,0,1,0


In [23]:
Yage_train.head()

0    34.5
1    47.0
2    62.0
3    27.0
4    22.0
Name: Age, dtype: float64

In [25]:
#creating test data for Age prediction
#for column where AGE
# Data to impute, -- Where Age is null; Remove completely-null "Age" column. taking all the null data seperate
impute=test[test.Age.isnull()].drop(['Age'], axis = 1)
print("Data to Impute")
impute.head(3)

Data to Impute


,Pclass,SibSp,Parch,Fare,Embarked_Q,Embarked_S,Sex_male
10,3,0,0,7.8958,0,1,1
22,1,0,0,31.6833,0,1,0
29,3,2,0,21.6792,0,0,1


In [27]:
#now modelling the new data with KNR
from sklearn.neighbors import KNeighborsRegressor
kr=KNeighborsRegressor()
kr.fit(Xage_train,Yage_train)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=1, n_neighbors=5, p=2,
          weights='uniform')

In [29]:
age_pred=kr.predict(impute)
age_pred

array([25.2  , 37.8  , 25.8  , 19.6  , 22.6  , 37.4  , 42.4  , 26.7  ,
       32.1  , 33.2  , 29.6  , 27.2  , 25.2  , 29.8  , 20.8  , 27.4  ,
       25.   , 27.2  , 26.7  , 26.7  , 22.6  , 27.4  , 23.4  , 26.7  ,
       26.7  , 19.6  , 26.384, 23.6  , 41.4  , 42.4  , 22.7  , 24.4  ,
       29.9  , 44.5  , 24.2  , 22.7  , 26.7  , 28.9  , 43.4  , 22.6  ,
       22.1  , 40.   , 29.9  , 25.6  , 27.2  , 16.266, 27.4  , 26.7  ,
       25.2  , 19.6  , 24.8  , 24.2  , 26.7  , 25.2  , 29.8  , 24.2  ,
       22.6  , 26.7  , 16.6  , 23.4  , 27.4  , 29.9  , 22.7  , 27.2  ,
       29.4  , 22.7  , 25.8  , 31.4  , 27.4  , 24.2  , 23.4  , 22.7  ,
       28.9  , 17.65 , 25.2  , 26.7  , 28.9  , 20.8  , 26.7  , 36.7  ,
       27.8  , 24.4  , 27.4  , 27.2  , 27.2  , 21.6  ])

In [31]:
#putting the predicted age column in Imputed(test) data
impute['Age']=age_pred
impute.head()

,Pclass,SibSp,Parch,Fare,Embarked_Q,Embarked_S,Sex_male,Age
10,3,0,0,7.8958,0,1,1,25.2
22,1,0,0,31.6833,0,1,0,37.8
29,3,2,0,21.6792,0,0,1,25.8
33,3,1,2,23.4500,0,1,0,19.6
36,3,0,0,8.0500,0,1,0,22.6


In [32]:
#concat the null data and non null data
new_test= pd.concat([non_null_tr, impute], sort = False, axis = 0)

# Return to original order - to match back up with "Survived"
new_test.sort_index(inplace = True)


In [35]:
#Our test data set is ready
new_test.head()

,Pclass,Age,SibSp,Parch,Fare,Embarked_Q,Embarked_S,Sex_male
0,3,34.5,0,0,7.8292,1,0,1
1,3,47.0,1,0,7.0000,0,1,0
2,2,62.0,0,0,9.6875,1,0,1
3,3,27.0,0,0,8.6625,0,1,1
4,3,22.0,1,1,12.2875,0,1,0


In [36]:
#same process for training data set
train.isna().sum()

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Embarked      2
dtype: int64

In [48]:

#Since sklearn converts all categorical data to float before encoding the model, converting all the cat data into dummy values
dummy_col=['Embarked','Sex']
train=pd.get_dummies(train,prefix=dummy_col, columns=dummy_col,drop_first=True )
train.head()



KeyError: "['Embarked' 'Sex'] not in index"

In [59]:
y_target=train.Survived
train_a=train.drop('Survived',axis=1)

In [60]:
train_a.isna().sum()

Pclass          0
Age           177
SibSp           0
Parch           0
Fare            0
Embarked_Q      0
Embarked_S      0
Sex_male        0
dtype: int64

In [61]:
#now working on Age data and predicting it through knr
#for age, since there are 86 values, a very high value
# so predicting age values through knr

#using non null data for training to get age
age_tr=train_a[train_a.Age.notnull()]
age_tr.isna().sum()

Pclass        0
Age           0
SibSp         0
Parch         0
Fare          0
Embarked_Q    0
Embarked_S    0
Sex_male      0
dtype: int64

In [62]:
#next step is to create training data out of it
#creating x, y datas for training and testing
xage_tr=age_tr.drop('Age',axis=1)
yage_tr=age_tr.Age

In [65]:
#test data
# Data to impute, -- Where Age is null; Remove completely-null "Age" column. taking all the null data seperate
age_imp=train_a[train_a.Age.isnull()].drop(['Age'], axis = 1)
print("Data to Impute")
age_imp.head(3)

Data to Impute


,Pclass,SibSp,Parch,Fare,Embarked_Q,Embarked_S,Sex_male
5,3,0,0,8.4583,1,0,1
17,2,0,0,13.0000,0,1,1
19,3,0,0,7.2250,0,0,0


In [66]:
knr=KNeighborsRegressor()
knr.fit(xage_tr,yage_tr)


KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=1, n_neighbors=5, p=2,
          weights='uniform')

In [67]:
imputed_age=knr.predict(age_imp)


array([47.2  , 29.4  , 23.   , 32.5  , 21.7  , 29.8  , 21.584, 21.1  ,
       24.8  , 27.8  , 30.2  , 30.8  , 21.1  , 26.4  , 49.4  , 38.9  ,
       16.2  , 29.8  , 30.2  , 21.1  , 30.2  , 30.2  , 29.8  , 34.2  ,
       32.6  , 30.2  , 47.2  , 17.6  , 20.4  , 34.8  , 26.8  , 37.   ,
       33.8  , 44.8  , 19.2  , 37.   , 32.   , 45.4  , 22.1  , 47.2  ,
       21.1  , 37.   , 47.2  , 29.8  , 21.4  , 25.8  , 19.9  , 22.1  ,
       34.8  , 39.2  , 47.2  , 21.1  , 48.6  , 21.1  , 33.8  , 42.6  ,
       38.9  , 48.6  , 21.1  , 29.6  , 33.6  , 30.2  , 28.   , 37.   ,
       23.2  , 40.4  , 29.8  , 27.2  , 49.4  , 32.5  , 21.7  , 21.7  ,
       30.8  , 22.   , 21.1  , 36.4  , 29.8  , 42.1  , 21.4  , 29.8  ,
       40.1  , 33.8  , 27.   , 27.8  , 34.8  , 47.2  , 27.2  , 21.6  ,
       27.   , 30.2  , 41.8  , 47.2  , 30.2  , 33.8  , 42.1  , 34.8  ,
       44.4  , 33.8  , 21.4  , 27.   , 22.4  , 30.3  , 21.1  , 52.6  ,
       30.2  , 32.2  , 32.5  , 24.8  , 30.8  , 24.8  , 14.   , 40.   ,
      

In [69]:
age_imp['Age']=imputed_age
age_imp.head()

,Pclass,SibSp,Parch,Fare,Embarked_Q,Embarked_S,Sex_male,Age
5,3,0,0,8.4583,1,0,1,47.2
17,2,0,0,13.0000,0,1,1,29.4
19,3,0,0,7.2250,0,0,0,23.0
26,3,0,0,7.2250,0,0,1,32.5
28,3,0,0,7.8792,1,0,0,21.7


In [70]:
#combining training and testing data together
#concat the null data and non null data
new_train= pd.concat([age_tr, age_imp], sort = False, axis = 0)

# Return to original order - to match back up with "Survived"
new_train.sort_index(inplace = True)


In [72]:
new_train.head()

,Pclass,Age,SibSp,Parch,Fare,Embarked_Q,Embarked_S,Sex_male
0,3,22.0,1,0,7.2500,0,1,1
1,1,38.0,1,0,71.2833,0,0,0
2,3,26.0,0,0,7.9250,0,1,0
3,1,35.0,1,0,53.1000,0,1,0
4,3,35.0,0,0,8.0500,0,1,1


In [73]:
#now predicting using logistic regression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
lr = LogisticRegression()
lr.fit(new_train,y_target)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [75]:
prediction=lr.predict(new_test)
prediction

array([0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0,
       1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1,
       1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1,
       1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,

In [91]:
pass_test_id.

0    892
1    893
2    894
3    895
4    896
Name: PassengerId, dtype: int64

In [95]:
solution=pd.concat([pass_test_id, pd.DataFrame({'Survived':prediction})], axis=1)

In [100]:
solution.reset_index()
solution.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1


In [101]:
solution.to_csv('/Users/hanyhamdani/Desktop/AppliedPython/machineLearning/TitanicKaggle/solution.csv')

In [102]:
sol_df=pd.read_csv('/Users/hanyhamdani/Desktop/AppliedPython/machineLearning/TitanicKaggle/solution.csv')

In [103]:
sol_df.head()

,Unnamed: 0,PassengerId,Survived
0,0,892,0
1,1,893,0
2,2,894,0
3,3,895,0
4,4,896,1


In [108]:
sol_df.drop(sol_df.columns[0],axis=1, inplace=True)
sol_df.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1


In [109]:
sol_df.index

RangeIndex(start=0, stop=418, step=1)

In [111]:
sol_df.to_csv('/Users/hanyhamdani/Desktop/AppliedPython/machineLearning/TitanicKaggle/sol.csv', index=False)